In [72]:
from mip import *
import json, time

In [60]:
file = open("./Instances/inst03.dat", "r")
lines = file.readlines()
file.close()
m = int(lines[0].rstrip(''))
n = int(lines[1].rstrip(''))
load = [int(x) for x in lines[2].split(' ')]
size = [int(x) for x in lines[3].split(' ')]
instance = [[int(x) for x in line.rstrip().split(' ') ]for line in lines[4:]]
o = n
print(f'Couriers: {m}')
print(f'Items: {n}')
print(f'Couriers Load: {load}')
print(f'Item\'s size: {size}')
print('Distance matrix:')
for i in range(n+1):
    for j in range(n+1):
        print(instance[i][j], end = " ")
    print()

Couriers: 3
Items: 7
Couriers Load: [15, 10, 7]
Item's size: [3, 2, 6, 8, 5, 4, 4]
Distance matrix:
0 3 3 6 5 6 6 2 
3 0 6 3 4 7 7 3 
3 4 0 7 6 3 5 3 
6 3 7 0 5 6 7 4 
5 4 6 3 0 3 3 3 
6 7 3 6 3 0 2 4 
6 7 5 6 3 2 0 4 
2 3 3 4 3 4 4 0 


In [4]:
# 2 variable model

# assignment[i,j] = 1 ---> item i assigned to courier j
assignment = [[model.add_var("assignment[%d,%d]" % (i, j), var_type=BINARY) for j in range(m)] for i in range(n)]

# tour[i][j] = y ---> courier i at j-th route has item y assigned
tour = [[model.add_var("tour_assignment[%d,%d]" % (i, j), var_type=INTEGER, lb=0, ub=n) for j in range(n+2)] for i in range(m)]

In [22]:
# 3 variable model
# assignment[i,j] = 1 ---> item i assigned to courier j
assignment = [[model.add_var("assignment[%d,%d]" % (i, j), var_type=BINARY) for j in range(m)] for i in range(n+2)]

# tour[i][j] = x ---> courier i dispatch at j-th route takes x
tour = [[model.add_var("tour[%d,%d]" % (i, j), var_type=INTEGER) for j in range(n+2)] for i in range(m)]

# tour_assignment[i][j] = y ---> courier i at j-th route has item y assigned
# tour_assignment = [[model.add_var("tour_assignment[%d,%d]" % (i, j), var_type=INTEGER, lb=0, ub=n) for j in range(n+2)] for i in range(m)]

# tour_assignment[i][j] = 1 ---> path from item i to item j is active
tour_assignment = [[model.add_var("tour_assignment[%d,%d]" % (i, j), var_type=BINARY, lb=0, ub=n) for j in range(n+2)] for i in range(n+1)]

# tour[c,i,j] == 1 ---> courier c performed movement from i to j
# tour = model.add_var_tensor((m, n+1, n+1), "tour", var_type=BINARY)

# Objective function
maxDist = model.add_var("maxDist", var_type=INTEGER)                                                                              
model.objective = maxDist

ValueError: Invalid bounds for binary variable

In [61]:
# Matrix model
model = Model(sense=MINIMIZE, solver_name=CBC)

# tour[c,i,j] == 1 ---> courier c performed movement from i to j
tour = model.add_var_tensor((m, n+1, n+1), "tour", var_type=BINARY)

# variable for subtours elimination
u = [model.add_var("u[%d]" % i, var_type=INTEGER, lb=0, ub=n) for i in range(n)]


maxDist = model.add_var("maxDist", var_type=INTEGER)
model.objective = maxDist

# Constraints for matrix model

# Each item is assigned to exactly one courier
for j in range(n):
    model += xsum([tour[c][i][j] for c in range(m) for i in range(n+1)]) == 1

# Each courier get only one time to the same item
for c in range(m):
    for i in range(n+1):
        model += xsum([tour[c][i][j] for j in range(n+1)]) == xsum([tour[c][j][i] for j in range(n+1)]) 

# Can't stay in the same spot
for c in range(m):
    for i in range(n+1):
        model += tour[c][i][i] == 0

# Each courier cannot load more than his capacity
for c in range(m):
    model += xsum([tour[c][i][j]*size[j] for i in range(n+1) for j in range(n)]) <= load[c]

# Initial and final destination are the same
for c in range(m):
    model += xsum([tour[c][n][j] for j in range(n)]) == 1
    model += xsum([tour[c][j][n] for j in range(n)]) == 1

# Miller-Tucker-Zemlin formulation for subtours elimination
for i in range(n):
    for j in range(n):
        for c in range(m):
            model += u[j] - u[i] >= 1 - n*(1 - tour[c][i][j])

distList = [xsum(instance[i][j]*tour[c][i][j] for i in range(n+1) for j in range(n+1)) for c in range(m)]
for c in range(m):
    model += maxDist >= distList[c]

In [62]:
def courier_tour(courier):
    done = False
    index = n
    out =[n+1]
    while not done:
        for j in range(n+1):
            if tour[courier][index][j].x:
                if j == n:
                    done = True
                out.append(j+1)
                break
        index = j
    return out

In [63]:
time_limit = 300


status = model.optimize(max_seconds=time_limit)

if status == OptimizationStatus.OPTIMAL:
    for c in range(m):
        print(f"Courier {c}")
        # out = []
        for i in range(n+1):
            print([tour[c][i][j].x for j in range(n+1)])
        #     for j in range(n+1):
        #         if tour[c][i][j].x :
        #             out.append(i+1)
        # print(out)
        print(f"TOUR: {courier_tour(c)}")
    print('optimal solution cost {} found'.format(model.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    elapsed_time = time_limit
else:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
            

Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 48 small elements
Coin0506I Presolve 166 (-48) rows, 176 (-24) columns and 1179 (-150) elements
Clp1000I sum of infeasibilities 9.1772e-06 - average 5.52844e-08, 14 fixed columns
Coin0506I Presolve 153 (-13) rows, 162 (-14) columns and 1072 (-107) elements
Clp0006I 0  Obj 12.738735 Dual inf 700 (1)
Clp0029I End of values pass after 162 iterations
Clp0014I Perturbing problem by 0.001% of 0.9417229 - largest nonzero change 2.3095329e-05 ( 0.0012317509%) - largest zero change 1.705264e-05
Clp0000I Optimal - objective value 10.514286
Clp0000I Optimal - objective value 10.514286
Coin0511I After Postsolve, objective 10.514286, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 10.514286
Clp0000I Optimal - objective value 10.514286
Clp0000I Optimal - objective value 10.514286
Clp0000I Optimal - objective value 10.514286
Coin0511I After Postsolve, objective 1

In [71]:
sol = [courier_tour(c) for c in range(m)]
type(status)

<enum 'OptimizationStatus'>